In [1]:
from collections import defaultdict
import pandas as pd
from bs4 import BeautifulSoup
import glob, os
import pickle

In [3]:
os.chdir("../../blogs") #see current directory
file_names = []
for file in glob.glob("*.xml"):
    file_names.append(file)

In [4]:
data = defaultdict()
for file_name in file_names:

    with open(file_name,encoding='utf-8',
                     errors='ignore') as fp:
        soup = BeautifulSoup(fp, 'xml')
        length = soup.find_all('date')
        contents_date = soup.find_all('date')
        contents_post = soup.find_all('post')
        temp = defaultdict()
        for i in range(len(length)):

            temp[contents_date[i].text] = contents_post[i].text

        data[file_name] = temp 

In [13]:
# for line in list:
#     if line[0] in years_dict:
#         # append the new number to the existing array at this slot
#         years_dict[line[0]].append(line[1])
#     else:
#         # create a new array in this slot
#         years_dict[line[0]] = [line[1]]

# POST BASED

In [5]:
x = pd.DataFrame([data]).T

In [6]:
x = x[0].apply(pd.Series)

In [7]:
x = pd.DataFrame(x.unstack()).reset_index().dropna(axis=0)

In [8]:
x.rename(columns={0:'post'}, inplace=True)

In [10]:
y = x['level_1'].str.split('.', expand=True)

In [11]:
x = pd.concat([x,y], axis=1)

In [12]:
x.sample(4)

,level_0,level_1,post,0,1,2,3,4,5
2078338,"26,January,2004",4161674.male.23.Student.Taurus.xml,"\n\n\n \n Input, output, pennies, d...",4161674,male,23,Student,Taurus,xml
539588,"26,July,2004",3974513.male.15.Automotive.Virgo.xml,\n\n\t \n It is a beautiful day today in...,3974513,male,15,Automotive,Virgo,xml
1046653,"02,August,2004",3312717.male.16.Student.Virgo.xml,\n\n\t \n Hello. There has been some int...,3312717,male,16,Student,Virgo,xml
1366981,"28,April,2004",2757248.female.15.Student.Gemini.xml,\n\n \n Z-sonny nut cheerios\n \n...,2757248,female,15,Student,Gemini,xml


In [13]:
x.rename(columns={'level_0':'date', 0:'blogger_id', 1:'gender', 2:'age', 3:'occupation', 4:'name'}, inplace=True)

In [14]:
x['post'] = x['post'].str.replace('\n','').str.replace('\t','').str.replace('urlLink', '')

In [15]:
x_fin = x[['date', 'blogger_id', 'gender', 'age', 'occupation', 'name', 'post']]

In [16]:
x_fin

,date,blogger_id,gender,age,occupation,name,post
0,"19,August,2004",4162441,male,16,Student,Sagittarius,DESTINY... you might not say a...
13,"19,August,2004",4157968,male,16,Student,Pisces,"As I strolled into the mall yesterday,..."
17,"19,August,2004",4278694,female,24,Technology,Virgo,I've never really had a blog bef...
69,"19,August,2004",4261114,female,40,indUnk,Gemini,I love to read novels. I l...
79,"19,August,2004",4215047,male,17,Student,Aquarius,today.. really a very sad day...
...,...,...,...,...,...,...,...
50463676,"12,February,1999",2999514,male,27,Engineering,Pisces,[detail] Propog...
50482996,"01,April,1999",2999514,male,27,Engineering,Pisces,[detail] I actually...
50502316,"05,June,1999",2999514,male,27,Engineering,Pisces,[detail] A tria...
50521636,"16,December,1999",2999514,male,27,Engineering,Pisces,[detail] Points...


In [252]:
with open('blogger_com_data_330677_7.pkl', 'wb') as pf:
    pickle.dump(x_fin, pf)

size of the dataset

In [21]:
d = x_fin['post'].apply(''.join)
d = ' '.join(d)
d = r.split()
len(d)

# USER BASED

In [30]:
user_based_x_fin = pd.DataFrame(x_fin.groupby(['blogger_id',
                                               'gender', 
                                               'age', 
                                               'occupation', 
                                               'name'])
                                ['post'].apply(' '.join))

In [31]:
user_based_x_fin.reset_index(inplace=True)

In [32]:
user_based_x_fin

,blogger_id,gender,age,occupation,name,post
0,1000331,female,37,indUnk,Leo,My four-year old never stops talking. ...
1,1000866,female,17,Student,Libra,"Yes, I did say goodbye to my friends an..."
2,1004904,male,23,Arts,Capricorn,if she wants me too then I will go to h...
3,1005076,female,25,Arts,Cancer,"The following things about last night ""..."
4,1005545,male,25,Engineering,Sagittarius,Blogdex Blogdex is a research pro...
...,...,...,...,...,...,...
19315,996147,female,36,Telecommunications,Leo,Ho hum.... Vacationed in Haw...
19316,997488,male,25,indUnk,Cancer,i've come back to blogs fire truck...
19317,998237,female,16,indUnk,Virgo,well hey look who it is... im actually...
19318,998966,male,27,indUnk,Taurus,"Finally, a green pepper is g..."


In [35]:
with open('blogger_com_data_19320_6_v2.pkl', 'wb') as pf:
    pickle.dump(user_based_x_fin, pf)